In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import log_loss
from PIL import ImageFile


In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

TRAIN_REAL_DIR = "train/real"   # Change this
TRAIN_FAKE_DIR = "train/fake"   # Change this
TEST_DIR = "test"               # Change this
#TEST_LABELS_PATH = "path/to/test_labels.csv"  # Ground truth (if available)

IMG_SIZE = (224, 224)  # Input size for CNN
BATCH_SIZE = 32


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.5, 1.5],
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    "train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    "train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"
)

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

print(f"Class Weights: {class_weights_dict}")


Found 38400 images belonging to 2 classes.
Found 9600 images belonging to 2 classes.
Class Weights: {0: 1.0, 1: 1.0}


In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation="relu", kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1, activation="sigmoid")  # Binary classification
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=["accuracy"])

history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=10,
                    class_weight=class_weights_dict)

final_train_accuracy = history.history['accuracy'][-1]
final_val_accuracy = history.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {final_train_accuracy:.4f}")
print(f"Final Validation Accuracy: {final_val_accuracy:.4f}")

model.save("fake_image_detector.h5")

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  74/1200 ━━━━━━━━━━━━━━━━━━━━ 2:13:00 7s/step - accuracy: 0.5266 - loss: 2.4088

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:3368: DecompressionBombWarning: Image size (90671520 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


1200/1200 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6215 - loss: 1.0123

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:3368: DecompressionBombWarning: Image size (99991727 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:3368: DecompressionBombWarning: Image size (98058240 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


1200/1200 ━━━━━━━━━━━━━━━━━━━━ 7106s 6s/step - accuracy: 0.6216 - loss: 1.0121 - val_accuracy: 0.6661 - val_loss: 0.6468
Epoch 2/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 6710s 6s/step - accuracy: 0.7165 - loss: 0.6053 - val_accuracy: 0.6875 - val_loss: 0.6313
Epoch 3/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 6701s 6s/step - accuracy: 0.7249 - loss: 0.5889 - val_accuracy: 0.6678 - val_loss: 0.6430
Epoch 4/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 6534s 5s/step - accuracy: 0.7311 - loss: 0.5788 - val_accuracy: 0.7011 - val_loss: 0.6113
Epoch 5/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 6509s 5s/step - accuracy: 0.7435 - loss: 0.5672 - val_accuracy: 0.7138 - val_loss: 0.5869
Epoch 6/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 6499s 5s/step - accuracy: 0.7467 - loss: 0.5651 - val_accuracy: 0.7181 - val_loss: 0.5976
Epoch 7/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 6501s 5s/step - accuracy: 0.7487 - loss: 0.5568 - val_accuracy: 0.7279 - val_loss: 0.5793
Epoch 8/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 6575s 5s/step - accuracy: 0.7583 - loss: 0.54

Final Training Accuracy: 0.7654
Final Validation Accuracy: 0.7302


In [ ]:
test_images = sorted(os.listdir(TEST_DIR))  # Ensure consistent order
predictions = []

for img_name in test_images:
    img_path = os.path.join(TEST_DIR, img_name)
    
    try:
        img = image.load_img(img_path, target_size=IMG_SIZE)
        img_array = image.img_to_array(img) / 255.0  # Normalize
        img_array = np.expand_dims(img_array, axis=0)

        pred_prob = model.predict(img_array)[0][0]  # Get probability
        binary_label = 1 if pred_prob > 0.5 else 0  # Convert to binary

        predictions.append([img_name, binary_label, pred_prob])  
    
    except Exception as e:
        print(f"Skipping corrupted image: {img_name} | Error: {e}")

submission_df = pd.DataFrame(predictions, columns=["filename", "binary_label", "logloss_label"])
submission_df.to_csv("submission_final.csv", index=False)
print("Submission file 'submission_final.csv' created successfully!")

try:
    ground_truth = pd.read_csv(TEST_LABELS_PATH)  # CSV with 'filename' & 'actual_label'
    merged = submission_df.merge(ground_truth, on="filename")
    logloss_score = log_loss(merged["actual_label"], merged["logloss_label"])
    print(f"LogLoss Score: {logloss_score}")
except Exception as e:
    print("Ground truth labels not found. LogLoss cannot be calculated.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━